In [19]:
import requests
import json
import os
from datetime import datetime
import praw
import pandas as pd
import time

In [26]:
# Capitalization does not matter in url
subreddit = 'offmychest'
# hot, new, top, controversial, rising
sort = 'hot'

start_time = datetime.now()

# Seems like reddit does not like someone crawling their website
# It will return 502 (Bad Gateway), but explanation is "Too Many Requests"

'''
# api url from web traffic
url = f'https://gateway.reddit.com/desktopapi/v1/subreddits/{subreddit}?sort={sort}'
res = requests.get(url)
print(res.text)
'''

# be polite and use APIs instead, unless...
from fake_useragent import UserAgent

FAKE_HEADERS = {'User-Agent': UserAgent().random}
url = f'https://gateway.reddit.com/desktopapi/v1/subreddits/{subreddit}?sort={sort}'

res = requests.get(url, headers=FAKE_HEADERS)
posts = json.loads(res.text)['posts']

output = []
for i in posts:
    output.append(posts[i]['title'])

output_text = '\n'.join(output)

# save
filename = 'output.txt'
metadata = {'subreddit': subreddit,
            'sort': sort,
            'time': str(start_time),
            'no_posts': len(output)
            }

with open('metadata.json', 'w') as file:
    file.write(json.dumps(metadata))

with open(filename, 'w') as file:
    file.write(output_text)

In [53]:
reddit = praw.Reddit(client_id='Ov0hf5hbdxVwAtI9CdodYw', client_secret='pzpqglx6N3_WoIAgJlMGsUowUkdwsw', user_agent='cz4045OMC')

start_time = time.time()

#https://towardsdatascience.com/scraping-reddit-data-1c0af3040768
while True:
    output = []
    n = 10
    t0= time.time()
    posts = reddit.subreddit('offmychest').new(limit=n)
    for post in posts:
        output.append([post.title, post.id])
    t1= time.time() - t0
    print(t1)

    df = pd.DataFrame(output, columns=['title', 'id'])
    data = pd.concat([df, data]).drop_duplicates('id').reset_index(drop=True)
    print('length:', len(data))
    data.to_csv('data.csv')
    time.sleep(60.0 - ((time.time() - start_time) % 60.0))

1.110440731048584
length: 990
0.4064173698425293
length: 990


KeyboardInterrupt: 